# Network Visualization with Bokeh

###  Importing Libraries

In [126]:
import pandas as pd
import networkx
import matplotlib.pyplot as plt
import numpy as np

### Loading the Data Set

In [127]:
df = pd.read_csv('./drug_gene2.csv')
# 데이터 불러오기

### Basic Data Exploration

In [128]:
df
# 데이터 살펴보기

,DRUG_NAME,TARGET_NAME,GENE,ORGANISM
0,levobupivacaine,Sodium channel protein type 4 subunit alpha,SCN4A,Homo sapiens
1,levobupivacaine,Cytochrome P450 2D6,CYP2D6,Homo sapiens
2,levobupivacaine,Potassium voltage-gated channel subfamily H me...,KCNH2,Homo sapiens
3,levobupivacaine,Potassium voltage-gated channel subfamily D me...,KCND3,Homo sapiens
4,levobupivacaine,Prostaglandin E2 receptor EP1 subtype,PTGER1,Homo sapiens
...,...,...,...,...
12106,niraparib,Poly [ADP-ribose] polymerase 2,PARP2,Homo sapiens
12107,ocrelizumab,B-lymphocyte antigen CD20,MS4A1,Homo sapiens
12108,dupilumab,Interleukin-4 receptor subunit alpha,IL4R,Homo sapiens
12109,deutetrabenazine,Synaptic vesicular amine transporter,SLC18A2,Homo sapiens


In [102]:
df['DRUG_NAME'].value_counts()

sunitinib           263
nintedanib          218
bosutinib           200
crizotinib          147
dasatinib           127
                   ... 
necitumumab           1
cyclophosphamide      1
benoxaprofen          1
lumefantrine          1
romiplostim           1
Name: DRUG_NAME, Length: 2087, dtype: int64

In [100]:
df['GENE'].value_counts()

HTR2A           173
ADRA2A          166
KCNH2           159
CHRM1           158
DRD3            158
               ... 
AOC3              1
HCRTR1            1
RDH12             1
IL11RA|IL6ST      1
PRKCI             1
Name: GENE, Length: 1497, dtype: int64

In [101]:
df[['DRUG_NAME','GENE']]

,DRUG_NAME,GENE
0,levobupivacaine,SCN4A
1,levobupivacaine,CYP2D6
2,levobupivacaine,KCNH2
3,levobupivacaine,KCND3
4,levobupivacaine,PTGER1
...,...,...
12106,niraparib,PARP2
12107,ocrelizumab,MS4A1
12108,dupilumab,IL4R
12109,deutetrabenazine,SLC18A2


In [105]:
result2 = result[['DRUG_NAME','GENE']].copy()
#DRUG_NAME 과 GENE 컬럼만 모아서 저장

In [106]:
result2.describe()

,DRUG_NAME,GENE
count,167,167
unique,122,29
top,bosutinib,SCN5A
freq,7,38


In [107]:
result2

,DRUG_NAME,GENE
6269,perhexiline,CPT2
7765,sunitinib,INSRR
10309,crizotinib,INSRR
11664,nintedanib,INSRR
351,amitriptyline,NTRK1
...,...,...
3410,floxacillin,MMP9
7225,secobarbital,MMP9
8890,zoledronic acid,MMP9
9355,quercetin,MMP9


## Importing Bokeh

In [108]:
G = networkx.from_pandas_edgelist(result2, 'GENE', 'DRUG_NAME')

In [109]:
from bokeh.io import output_notebook, show, save
from bokeh.models import Range1d, Circle, ColumnDataSource, MultiLine
from bokeh.plotting import figure
from bokeh.plotting import from_networkx

In [110]:
# output_notebook()

## Basic Network

In [134]:
#Choose a title!
title = 'Drug-Gene Interaction'

#Establish which categories will appear when hovering over each node
HOVER_TOOLTIPS = [("Character", "@index")]

#Create a plot — set dimensions, toolbar, and title
plot = figure(tooltips = HOVER_TOOLTIPS,
              tools="pan,wheel_zoom,save,reset", active_scroll='wheel_zoom',
            x_range=Range1d(-10.1, 10.1), y_range=Range1d(-10.1, 10.1), title=title) #plot_width=1500, plot_height=1500 크기조절

#Create a network graph object with spring layout
network_graph = from_networkx(G, networkx.spring_layout, scale=10, center=(0, 0))

#Set node size and color
network_graph.node_renderer.glyph = Circle(size=15, fill_color='skyblue')

#Set edge opacity and width
network_graph.edge_renderer.glyph = MultiLine(line_alpha=0.5, line_width=1)

#Add network graph to the plot
plot.renderers.append(network_graph)

show(plot)
#save(plot, filename=f"{title}.html")

## Network with Nodes Sized and Colored By Attribute (Degree)

In [115]:
from bokeh.io import output_notebook, show, save
from bokeh.models import Range1d, Circle, ColumnDataSource, MultiLine
from bokeh.plotting import figure
from bokeh.plotting import from_networkx
from bokeh.palettes import Blues8, Reds8, Purples8, Oranges8, Viridis8, Spectral8
from bokeh.transform import linear_cmap

In [116]:
degrees = dict(networkx.degree(G))
networkx.set_node_attributes(G, name='degree', values=degrees)

In [117]:
number_to_adjust_by = 5
adjusted_node_size = dict([(node, degree+number_to_adjust_by) for node, degree in networkx.degree(G)])
networkx.set_node_attributes(G, name='adjusted_node_size', values=adjusted_node_size)

In [118]:
#Choose attributes from G network to size and color by — setting manual size (e.g. 10) or color (e.g. 'skyblue') also allowed
size_by_this_attribute = 'adjusted_node_size'
color_by_this_attribute = 'adjusted_node_size'

#Pick a color palette — Blues8, Reds8, Purples8, Oranges8, Viridis8
color_palette = Blues8

#Choose a title!
title = 'Drug-Gene Interaction'

#Establish which categories will appear when hovering over each node
HOVER_TOOLTIPS = [
       ("Character", "@index"),
        ("Degree", "@degree")
]

#Create a plot — set dimensions, toolbar, and title
plot = figure(tooltips = HOVER_TOOLTIPS,
              tools="pan,wheel_zoom,save,reset", active_scroll='wheel_zoom',
            x_range=Range1d(-10.1, 10.1), y_range=Range1d(-10.1, 10.1), title=title) #plot_width=1500, plot_height=1500 크기조절

#Create a network graph object
network_graph = from_networkx(G, networkx.spring_layout, scale=10, center=(0, 0))

#Set node sizes and colors according to node degree (color as spectrum of color palette)
minimum_value_color = min(network_graph.node_renderer.data_source.data[color_by_this_attribute])
maximum_value_color = max(network_graph.node_renderer.data_source.data[color_by_this_attribute])
network_graph.node_renderer.glyph = Circle(size=size_by_this_attribute, fill_color=linear_cmap(color_by_this_attribute, color_palette, minimum_value_color, maximum_value_color))

#Set edge opacity and width
network_graph.edge_renderer.glyph = MultiLine(line_alpha=0.5, line_width=1)

plot.renderers.append(network_graph)

show(plot)
#save(plot, filename=f"{title}.html")

## Network with Responsive Highlighting

In [119]:
from bokeh.io import output_notebook, show, save
from bokeh.models import Range1d, Circle, ColumnDataSource, MultiLine, EdgesAndLinkedNodes, NodesAndLinkedEdges
from bokeh.plotting import figure
from bokeh.plotting import from_networkx
from bokeh.palettes import Blues8, Reds8, Purples8, Oranges8, Viridis8, Spectral8
from bokeh.transform import linear_cmap
from networkx.algorithms import community

In [120]:
degrees = dict(networkx.degree(G))
networkx.set_node_attributes(G, name='degree', values=degrees)

In [121]:
number_to_adjust_by = 5
adjusted_node_size = dict([(node, degree+number_to_adjust_by) for node, degree in networkx.degree(G)])
networkx.set_node_attributes(G, name='adjusted_node_size', values=adjusted_node_size)

In [122]:
communities = community.greedy_modularity_communities(G)

In [123]:
# # Create empty dictionaries
# modularity_class = {}
# modularity_color = {}
# #Loop through each community in the network
# for community_number, community in enumerate(communities):
#     #For each member of the community, add their community number and a distinct color
#     for name in community: 
#         modularity_class[name] = community_number
#         modularity_color[name] = Spectral8[community_number]

In [124]:
# Add modularity class and color as attributes from the network above
networkx.set_node_attributes(G, modularity_class, 'modularity_class')
networkx.set_node_attributes(G, modularity_color, 'modularity_color')

## Network with Labels

In [136]:
from bokeh.models import EdgesAndLinkedNodes, NodesAndLinkedEdges

#Choose colors for node and edge highlighting
node_highlight_color = 'white'
edge_highlight_color = 'black'

#Choose attributes from G network to size and color by — setting manual size (e.g. 10) or color (e.g. 'skyblue') also allowed
size_by_this_attribute = 'adjusted_node_size'
color_by_this_attribute = 'modularity_color'

#Pick a color palette — Blues8, Reds8, Purples8, Oranges8, Viridis8
color_palette = Blues8

#Choose a title!
title = 'Drug-Gene Interaction'

#Establish which categories will appear when hovering over each node
HOVER_TOOLTIPS = [
       ("Character", "@index"),
        ("Degree", "@degree"),
         ("Modularity Class", "@modularity_class"),
        ("Modularity Color", "$color[swatch]:modularity_color"),
]

#Create a plot — set dimensions, toolbar, and title
plot = figure(tooltips = HOVER_TOOLTIPS,
              tools="pan,wheel_zoom,save,reset", active_scroll='wheel_zoom',
            x_range=Range1d(-10.1, 10.1), y_range=Range1d(-10.1, 10.1), title=title) #plot_width=1500, plot_height=1500 크기조절

#Create a network graph object
network_graph = from_networkx(G, networkx.spring_layout, scale=10, center=(0, 0))

#Set node sizes and colors according to node degree (color as category from attribute)
network_graph.node_renderer.glyph = Circle(size=size_by_this_attribute, fill_color=color_by_this_attribute)
#Set node highlight colors
network_graph.node_renderer.hover_glyph = Circle(size=size_by_this_attribute, fill_color=node_highlight_color, line_width=2)
network_graph.node_renderer.selection_glyph = Circle(size=size_by_this_attribute, fill_color=node_highlight_color, line_width=2)

#Set edge opacity and width
network_graph.edge_renderer.glyph = MultiLine(line_alpha=0.5, line_width=1)
#Set edge highlight colors
network_graph.edge_renderer.selection_glyph = MultiLine(line_color=edge_highlight_color, line_width=2)
network_graph.edge_renderer.hover_glyph = MultiLine(line_color=edge_highlight_color, line_width=2)

#Highlight nodes and edges
network_graph.selection_policy = NodesAndLinkedEdges()
network_graph.inspection_policy = NodesAndLinkedEdges()

plot.renderers.append(network_graph)

#Add Labels
x, y = zip(*network_graph.layout_provider.graph_layout.values())
node_labels = list(G.nodes())
source = ColumnDataSource({'x': x, 'y': y, 'name': [node_labels[i] for i in range(len(x))]})
labels = LabelSet(x='x', y='y', text='name', source=source, background_fill_color='white', text_font_size='10px', background_fill_alpha=.7)
plot.renderers.append(labels)

show(plot)
#save(plot, filename=f"{title}.html")

In [ ]:
## reference
## https://melaniewalsh.github.io/Intro-Cultural-Analytics/welcome.html